## Homework 04
### Math 5364 - Data Mining 1, Fall 2017, Tarleton State Univ, Dr. Scott Cook
### Due Friday 10/27/2017 at midnight

The purpose of this homework is to explore Naive Bayes and cross-validation.
- Get the House Votes 84 data set from the UCI machine learning archive
1. Write a Naive Bayes Classifier to predict party affliation from the voting record of a congress person.
  - Though NB is one of our simpler classifiers, this is still a non-trivial task.  I'll give you substantial starter code below.
  - You may use the MultinomialNB classifier in Scikit learn to check your work.  Note that you may not get *exactly* the same predictions and posterior because scikit learn implements some more advanced refinements, like Laplace smoothing (in the alpha parameter).
- Train and test with the entire data set at first, as you validate your code against scikit learn.
2. Now, write code to implement $k$-fold cross validation for *your* NB classifier.
  - From now on, set seed = 42, for convenient comparisions with your classmates.
  - Create a plot:
    - Let $k$ range over the integers $5 \leq k \leq 145$ that divide 435.  Note that $435/k$ = number of obervations in the testing set; it ranges from 87 to 3.  Plot $435/k$ on the horizontal axis.
    - mean accuracy score when using that value of $k$ on the vertical.
3. Now, write code to implement delete-$d$ cross validation for *your* NB classifier.  Let $d$ range over the integers $3 \leq d \leq 87$ that divide 435.  Repeat $b=10$ times for each value of $d$.  Create a similar plot as above with $d$ on the the horizontal.
4. Now write code to implement bootstap cross-validation.  (Recall, unlike the prior methods, this one samples *with* replacement).  Here, we always resample $n=435$ times to make the training set, and test on the observations that never were selected.  Let $b=100$ be the numbers of validation cycles.  Compute accuracy in 2 ways
  - As normal: Compute accuracy on the testing set for each bootrap sample, then take the mean.
  - Using the 632 correction discussed
    - Original source - http://web.cs.iastate.edu/~jtian/cs573/Papers/Kohavi-IJCAI-95.pdf
    - More readable exposition - http://www3.cs.stonybrook.edu/~cse634/lecture_notes/07testing.pdf
    - Many other places - make use of reliable internet resources
  - Evaluate the 632 correction in light of your results.

In [7]:
from setup import *

na_legal = True

df = pd.read_csv('house-votes-84.csv', header=-1)
if(na_legal == True):
    df.replace('?','z', inplace=True)
else:
    df.replace('?' ,np.nan, inplace=True)

X_orig = df.loc[:,1:]
X = pd.get_dummies(X_orig)
Y = df.loc????
labels = sorted(Y.unique())

def vc(x):
    return x.value_counts(normalize=True)
prior = vc(Y)

def f(grp):
    return pd.concat([vc(col[1]) for col in grp.items()], ????)

marginals = X_orig.groupby(Y).????.unstack()
#Next line makes columns names in marginals match those in X
marginals.columns = ['_'.join(map(str,col)) for col in marginals.columns]

# NB demands that we multiply the marginals.  But, it is a bad idea to multiplt
# a bunch of number together in a computer.  If they are big, the product can overflow
# and create errors.  If the numbers are small, they can drop into "rounding error" and 
# again create errors.  So, when we are asked to multiply a bunch of numbers, we often use
# the log trick.  Recall that the log of things multiplied equals the sum of their logs.
# So, we do a*b*c*d...*z = e^(ln(a*b*c*d*...*z)) = e^(ln(a)+ln(b)+ln(c)+ln(d)+...+ln(z)).
# The sum of logs does not grow/shrink as fast as the product, so we often avoid
# overflow/rounding errors.
log_marginals = ????

# Picks out the numbers we need from log_marginals
log_posterior = ????.dot(????.T)
log_posterior = log_posterior + ????
posterior = np.exp(log_posterior)
# normalize
posterior = posterior.div(posterior.sum(axis=1), axis=0)
pred = posterior.idxmax(axis=1)

my_posterior = posterior.copy()
my_pred = pred.copy()


from sklearn.naive_bayes import MultinomialNB
# Another reason our NB did not match sk_learns NB is that sk_learn uses a
# "smoothing" process.  Setting alpha = 0 turns that off.  Now, that process may
# be very helpful.  I'm turning it off only to show that sk_learn and our code agree.
model = MultinomialNB(alpha=0.0)
model.????
skl_posterior = model.predict_proba(????)
skl_posterior = pd.DataFrame(skl_posterior,columns=labels) 
skl_pred = model.predict(X)

from sklearn.metrics import confusion_matrix
def assess(pred):
    mat = confusion_matrix(Y, pred, labels)
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=labels, yticklabels=labels)
    plt.xlabel('true label')
    plt.ylabel('predicted label')
    plt.show()    
    print("accuracy = %f",mat.????.sum()/mat.sum()*100)

assess(my_pred)
assess(skl_pred)

    
cls = pd.DataFrame()
cls['true'] = Y.copy()
cls['my_pred'] = my_pred.copy()
cls['im_right'] = ????
cls = pd.concat([cls,my_posterior],axis=1)

cls['skl_pred'] = skl_pred.copy()
cls['skl_right'] = ????
cls = pd.concat([cls,skl_posterior],axis=1)

cls['pred_same'] = ????
cls['close_call'] = (np.abs(my_posterior.iloc[:,0] - my_posterior.iloc[:,1]) <= 0.4)
diffus = np.linalg.norm(my_posterior-skl_posterior, axis=1)
biggus_diffus = np.argsort(diffus)[::-1][:5]

print("Skl and I made different predictions")
display(cls.loc[????,:],'all')
print("I'm wrong")
display(cls.loc[????,:],'all')
print("Close call")
display(cls.loc[cls['close_call'],:],'all')
print("Biggus Diffus between skl and me")
display(cls.loc[biggus_diffus,:],'all')

SyntaxError: invalid syntax (<ipython-input-7-0a61f8a5462e>, line 13)